# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import spacy
import matplotlib.pyplot as plt
import os
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM
import torch
import re
import tensorflow as tf
from models.llama3.generation import Llama

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_colwidth', None)  # Show full content in each cell
pd.set_option('display.width', 1000)  # Set max width

# Load spaCy's English model
nlp = spacy.load('en_core_web_sm')

/opt/anaconda3/envs/yt_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Question Classifier

## Text-Classification

In [16]:
tokenizer = AutoTokenizer.from_pretrained("uw-vta/bloominzer-0.1")
model = AutoModelForSequenceClassification.from_pretrained("uw-vta/bloominzer-0.1")
bloominzer = pipeline("text-classification", model=model, tokenizer=tokenizer)

Device set to use mps:0


In [ ]:
print(bloominzer("If I have 2 pair of apple, can i make apple pie with it?"))

Device set to use mps:0


[{'label': 'Synthesis', 'score': 0.9990537762641907}]


## LLM Classification

## Zero-Shot

### BART

In [8]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-mnli")
model = AutoModelForSequenceClassification.from_pretrained("facebook/bart-large-mnli")

bart = pipeline("zero-shot-classification",
                      model=model , tokenizer=tokenizer)

Device set to use mps:0


RuntimeError: MPS backend out of memory (MPS allocated: 20.34 GB, other allocations: 5.95 MB, max allowed: 20.40 GB). Tried to allocate 196.35 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
sequence_to_classify = "If I have 2 pair of apple, can i make apple pie with it?"
candidate_labels = ['knowledge', 'comprehension', 'application', 'analysis','synthesis', 'evaluation']
label = bart(sequence_to_classify, candidate_labels)

In [15]:
label['labels'][0]

'application'

### mDeBERTa-v3-base-mnli-xnli

In [15]:
tokenizer = AutoTokenizer.from_pretrained("MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")
model = AutoModelForSequenceClassification.from_pretrained("MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")

ya_classifier = pipeline("zero-shot-classification",
                      model=model , tokenizer=tokenizer)

Device set to use mps:0


In [16]:
sequence_to_classify = "If I have 2 pair of apple, can i make apple pie with it?"
candidate_labels = ['knowledge', 'comprehension', 'application', 'analysis','synthesis', 'evaluation']
hypothesis_template = "This text is about blooms taxonomy and it is classified as {}."
label = ya_classifier(sequence_to_classify, candidate_labels, hypothesis_template=hypothesis_template)

In [17]:
label['labels'][0]

'knowledge'

## Text Generation

### LLAMA

In [3]:
from llama_cpp import Llama

llm = Llama.from_pretrained(
	repo_id="modularai/Llama-3.1-8B-Instruct-GGUF",
	filename="llama-3.1-8b-instruct-q4_k_m.gguf"
)


llama_model_load_from_file_impl: using device Metal (Apple M3 Pro) - 12287 MiB free
llama_model_loader: loaded meta data with 33 key-value pairs and 292 tensors from /Users/pranitdas/.cache/huggingface/hub/models--modularai--Llama-3.1-8B-Instruct-GGUF/snapshots/966694508430d1177f6d585de779250e7a34bc3a/./llama-3.1-8b-instruct-q4_k_m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Meta Llama 3.1 8B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Meta-Llama-3.1
llama_model_l

In [ ]:
query = 'If I have 2 pair of apple, can i make apple pie with it?'

llm.create_chat_completion(
    messages=[
        {
            "role": "user",
            "content": f"""
            Classify this query's Bloom's taxanomy level using ONLY one word from: 
            [knowledge, comprehension, application, analysis, synthesis, evaluation]

            query : {query}

            Context: Bloom's taxonomy with examples:
            1. **Knowledge** (Recall facts): 
               - "Name three Shakespeare plays"
               - "What year did WWII end?"
               
            2. **Comprehension** (Understand meaning):
               - "Explain how photosynthesis works"
               - "Summarize the main theme of 1984"
               
            3. **Application** (Use concepts):
               - "Calculate the interest on a $5000 loan"
               - "How would you apply Newton's laws to this scenario?"
               
            4. **Analysis** (Examine structure):
               - "Compare Marxist vs capitalist economic models"
               - "Identify logical fallacies in this argument"
               
            5. **Synthesis** (Create new patterns):
               - "Design an experiment to test water purity"
               - "Propose a city planning solution for traffic"
               
            6. **Evaluation** (Make judgments):
               - "Which algorithm is most efficient for this task?"
               - "Assess the ethical implications of AI surveillance"

            Decision Framework:
            1. Identify the CORE ACTION:
               - Remember/List → Knowledge
               - Explain/Paraphrase → Comprehension
               - Apply/Use/Calculate → Application
               - Compare/Contrast → Analysis
               - Create/Design → Synthesis
               - Judge/Recommend → Evaluation

            2. Ask: Does the query require...
               - Simple recall? → Knowledge
               - Basic understanding? → Comprehension
               - Practical implementation? → Application
               - Breaking into components? → Analysis
               - Original construction? → Synthesis
               - Critical assessment? → Evaluation

            Respond ONLY with the exact taxonomy level name. No explanations.
            """
        }
    ]
)

llama_perf_context_print:        load time =    6889.15 ms
llama_perf_context_print: prompt eval time =    6888.89 ms /   433 tokens (   15.91 ms per token,    62.85 tokens per second)
llama_perf_context_print:        eval time =      58.27 ms /     1 runs   (   58.27 ms per token,    17.16 tokens per second)
llama_perf_context_print:       total time =    6948.07 ms /   434 tokens


{'id': 'chatcmpl-991d46d0-d2af-4e1e-a57d-77f9a71d5c87',
 'object': 'chat.completion',
 'created': 1748259754,
 'model': '/Users/pranitdas/.cache/huggingface/hub/models--modularai--Llama-3.1-8B-Instruct-GGUF/snapshots/966694508430d1177f6d585de779250e7a34bc3a/./llama-3.1-8b-instruct-q4_k_m.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant', 'content': 'Analysis'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 433, 'completion_tokens': 1, 'total_tokens': 434}}

### DeepSeek

In [2]:
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

r1_classifier = pipeline("text-generation", model=model , tokenizer=tokenizer)

Device set to use mps:0


In [14]:
import re

query = 'What is the capital of india?'

# Step 1: Ask for classification
messages = [
    {
        "role": "user", 
        "content": f"""
Classify this query's Bloom's taxonomy level using ONLY one word from: 
[knowledge, comprehension, application, analysis, synthesis, evaluation]
Respond with only one word.
query: {query}""",
    }
]
data = r1_classifier(messages)

# Step 2: Extract assistant's response
assistant_content = next(
    item['content'] for item in data[0]['generated_text'] 
    if item['role'] == 'assistant'
).strip().lower()

# Step 3: Extract the classification using regex
bloom_keywords = ['knowledge', 'comprehension', 'application', 
                 'analysis', 'synthesis', 'evaluation']

classification = re.search(
    r'\b(' + '|'.join(bloom_keywords) + r')\b',
    assistant_content,
    flags=re.IGNORECASE
).group(1).lower()

print(classification)

knowledge


In [12]:
query = 'What is the capital of india?'

messages = [
    {
        "role": "user", 
        "content": f'''
        Classify this query's Bloom's taxanomy level using ONLY one word from: 
        [knowledge, comprehension, application, analysis, synthesis, evaluation]
        query: {query}''',
    }
]
data = r1_classifier(messages)

assistant_content = next(
    item['content'] for item in data[0]['generated_text'] if item['role'] == 'assistant'
)

messages = [
    {
        "role": "user", 
        "content": f'''
        Extract one word bloom's taxanomy level from the response: 
        [knowledge, comprehension, application, analysis, synthesis, evaluation]
        response: {assistant_content}''',
    }
]

r1_classifier(messages)

[{'generated_text': [{'role': 'user',
    'content': '\n        Extract one word bloom\'s taxanomy level from the response: \n        [knowledge, comprehension, application, analysis, synthesis, evaluation]\n        response: Okay, so I need to figure out the Bloom\'s taxonomic level for the query "What is the capital of India?" using only one word from the options: knowledge, comprehension, application, analysis, synthesis, evaluation.\n\nFirst, let me recall what each level means. Bloom\'s taxonomic levels are a way to classify educational objectives and are used to assess the complexity of questions. The levels go from simple to complex, and each level represents a higher order of thinking.\n\n1. **Knowledge**: This is the lowest level, where the question is asking for information or data. For example, "What is the capital of India?" is asking for factual information.\n\n2. **Comprehension**: This is a bit higher. It involves understanding the meaning of the question. The answer wou

# Query Augmentation